In [ ]:
import ee, h5py, warnings
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

In [ ]:
# Trigger the authentication flow.
ee.Authenticate()

In [ ]:
# Initialize the library.
ee.Initialize()

In [ ]:
elv_id = xr.open_dataset('/content/elevation_0.25deg.nc')

elv_id = elv_id.sel(latitude = slice(20, 90))

elv_id = elv_id['elev'][0,0,:,:]

elv_id

In [ ]:
# modify the elevation matrix to match GDDP data ...

elv_values = np.transpose(elv_id.values)
elv_values[0:720]
elv_values[720:]

elv_values = np.r_[elv_values[720:], elv_values[0:720]]


In [ ]:
plt.imshow(elv_values)

In [ ]:
gee_id = ee.ImageCollection("NASA/GDDP-CMIP6")

var_list = ['tas','pr','hurs','sfcWind']

In [ ]:
gee_poi = ee.Geometry.BBox(-180, 20, 180, 90)

In [ ]:
year0 = 1984

In [ ]:
i_date = str(year0)+"-01-01"
f_date = str(year0)+"-01-02"

In [ ]:
gee_id_2 = gee_id.select(['tas']).filterDate(i_date, f_date).filter(ee.Filter.eq('model', 'ACCESS-CM2'))

In [ ]:
gee_data = gee_id_2.getRegion(gee_poi, 27830 * 1).getInfo()

In [ ]:
df = pd.DataFrame(gee_data)
# Rearrange the header.
headers = df.iloc[0]
df = pd.DataFrame(df.values[1:], columns=headers)

In [ ]:
df

In [ ]:
len(df.latitude.unique())
len(df.longitude.unique())

In [ ]:
test = df.pivot('longitude','latitude')['tas'].values

test[test == None] = np.nan

test = np.float16(test)

test

In [ ]:
plt.imshow(test)

In [ ]:
plt.scatter(df.longitude, df.latitude)

In [ ]:
df = df[df.tas.values != None]
df = df[df.pr.values != None]